In [11]:
import pickle
import os

In [55]:
dump = pickle.load(open('mem_obj.dump', 'rb'))
len(dump)
for obj, count_size in zip(dump.keys(), dump.values()):
    if '[' in obj:
        print(obj)
        print(count_size)
        print('-' * 80)

[anon:.bss]
[61, 19664896]
--------------------------------------------------------------------------------
[anon:libc_malloc]
[32, 270532608]
--------------------------------------------------------------------------------
[anon:thread
[325, 2670592]
--------------------------------------------------------------------------------
[anon:bionic
[324, 2232320]
--------------------------------------------------------------------------------
[anon:linker_alloc_vector]
[6, 28672]
--------------------------------------------------------------------------------
[anon:linker_alloc]
[69, 323584]
--------------------------------------------------------------------------------
[anon:atexit
[12, 53248]
--------------------------------------------------------------------------------
[anon:linker_alloc_small_objects]
[14, 65536]
--------------------------------------------------------------------------------
[anon:cfi
[12, 2147483648]
-----------------------------------------------------------------

In [3]:
sum_size = 0
for obj, count_size in zip(dump.keys(), dump.values()):
    if obj == '':
        continue
    wrong_words = {'dalvik', 'boot', 'font', 'Chrome', 'kgsl', 'system', 'lib', 'Audio', 'System', '.dat', '/dev/__properties__', 'dex', 'apk', 'web', 'config', 'cfi', 'binder', '.bss', 'thread', 'bionic'}
    good = True
    for word in wrong_words:
        if word in obj:
            good = False
            break
    if good:
        print(obj)
        print(count_size)
        print('-' * 80)
        sum_size += count_size[1]
print(sum_size)

/dev/ashmem/shared_memory/D5A74A121842A47E7EAF68382AD3794C
[1, 131072]
--------------------------------------------------------------------------------
/dev/ashmem/shared_memory/1D53F3F180B6B1A832373957D4805C13
[1, 28672]
--------------------------------------------------------------------------------
anon_inode:dmabuf
[3, 12288]
--------------------------------------------------------------------------------
[anon:linker_alloc_vector]
[6, 28672]
--------------------------------------------------------------------------------
[anon:linker_alloc]
[69, 323584]
--------------------------------------------------------------------------------
[anon:atexit
[12, 53248]
--------------------------------------------------------------------------------
[anon:linker_alloc_small_objects]
[14, 65536]
--------------------------------------------------------------------------------
/data/data/com.innogames.riseofcultures/no_backup/androidx.work.workdb-shm
[1, 32768]
-----------------------------------

In [6]:
dumps = list()
for i in range(11):
    dump = pickle.load(open(f'{i:03d}.dump', 'rb'))
    for key in dump:
        if '[stack]' in key[2]:
            dumps.append(dump)
            break

In [7]:
exists_in_all_dumps = list()
dump_0 = dumps[0]
for key in dump_0.keys():
    in_all = True
    for dump in dumps[1:]:
        if key not in dump:
            in_all = False
            break
    if in_all:
        exists_in_all_dumps.append(key)
print(f'{len(exists_in_all_dumps)} записей сохраняется по всем дампам')

114 записей сохраняется по всем дампам


In [8]:
const_in_all_dumps = list()
for key in exists_in_all_dumps:
    in_all = True
    for dump in dumps[1:]:
        if dump_0[key] != dump[key]:
            in_all = False
            break
    if in_all:
        const_in_all_dumps.append(key)
print(f'{len(const_in_all_dumps)} записей не меняют содержания по всем дампам')

109 записей не меняют содержания по всем дампам


In [9]:
diff_mem = list()
for key in exists_in_all_dumps:
    if key not in const_in_all_dumps:
        diff_mem.append(key)
print('Следующие записи меняются от переговоров к переговорам:')
for key in diff_mem:
    print(key)

Следующие записи меняются от переговоров к переговорам:
('0x7dcf1ea000', 4096, '/dev/ashmem/b390658b-3797-4724-bafe-266be4332e9d')
('0x7e502e3000', 4096, '/dev/ashmem/b390658b-3797-4724-bafe-266be4332e9d')
('0x7e5042c000', 4096, '/dev/ashmem/b390658b-3797-4724-bafe-266be4332e9d')
('0x7e554d5000', 4096, '[vvar]')
('0x7fce384000', 8384512, '[stack]')


In [10]:
'''
Резонно, ведь /dev/ashmem это куча, а стэк не может не меняться.
Хотя на куче меняются очень маленькие аллокации.
Содержимое vvar принадлежит ядру и не может меняться пользовательским кодом.
Стоит сначала исследовать стек.
'''

'\nРезонно, ведь /dev/ashmem это куча, а стэк не может не меняться.\nХотя на куче меняются очень маленькие аллокации.\nСодержимое vvar принадлежит ядру и не может меняться пользовательским кодом.\nСтоит сначала исследовать стек.\n'

In [13]:
stacks = list()
try:
    os.mkdir('./stack/')
except:
    pass
for i, dump in zip(range(len(dumps)), dumps):
    for key in dump.keys():
        if '[stack]' in key[2]:
            stacks.append(dump[key])
            open(f'./stack/{i:03d}.dump', 'wb').write(dump[key])
            break

In [31]:
from difflib import SequenceMatcher as sm

def bytes_to_str(data):
    return ' '.join(['%.2X' % (x,) for x in data])

In [34]:
offset_7MB = 7340032
diff = sm(None, stacks[0][offset_7MB:], stacks[2][offset_7MB:])

In [37]:
status = set()
begin = 0
end = 0
begins = None
lengths = dict()
offset_7MB = 7340032
for stack in stacks[1:]:
    diff = sm(None, stacks[0][offset_7MB:], stack[offset_7MB:])
    new_begins = set()
    for opcode in diff.get_opcodes():
        #print("%7s a[%d:%d] b[%d:%d]" % opcode)
        if opcode[0] == 'equal' and opcode[1] == opcode[3] and opcode[2] == opcode[4]:
            end = opcode[1]
            #if begin != 0:
            #    print(f'{begin:8d}{end:8d} -> {end - begin:8d}')
            new_begins.add(begin)
            length = end - begin
            if begin in lengths:
                lengths[begin].add(length)
            else:
                lengths[begin] = {length}
            begin = opcode[2]
    if not begins:
        begins = new_begins
    else:
        begins = begins.intersection(new_begins)
for begin in begins:
    print(f'{begin} -> {lengths[begin]}')

1033476 -> {1, 2}
1018896 -> {1, 2, 3}
1021712 -> {2, 3}
1033744 -> {11, 3, 7}
1033496 -> {1, 6, 7}
1021080 -> {5}
1019936 -> {2}
1033376 -> {1, 2}
1021352 -> {2, 3}
1033772 -> {2, 3}
1016496 -> {2}
1019056 -> {1, 2, 3}
1020592 -> {4}
1019696 -> {1, 2, 3}
1020720 -> {2, 3}
1021232 -> {2, 3}
1019700 -> {1, 2, 3}
1033524 -> {3}
1021120 -> {2, 3}
1014465 -> {11}
1019712 -> {3}
1020224 -> {1}
1019208 -> {1, 2, 3}
1019848 -> {1, 2, 3}
1020369 -> {4}
1014488 -> {117}
1014616 -> {16}
1021784 -> {1, 2, 3}
1018840 -> {1, 2, 3}
1021912 -> {2, 3}
1021916 -> {1, 2, 3}
1020389 -> {43}
1016424 -> {3}
1020520 -> {4, 11, 18, 19, 23, 24, 63}
1018600 -> {2, 3}
1033192 -> {1, 2}
1020652 -> {1}
1022189 -> {6136, 10832, 5958, 5959}
1014640 -> {1}
1014642 -> {800, 801, 742, 1622, 862}
1018744 -> {2, 3}


In [39]:
for stack in stacks:
    print(bytes_to_str(stack[offset_7MB + 1021080: offset_7MB + 1021080 + 5]))

00 4C 61 CF 7D
58 F0 B7 CE 7F
58 F0 B7 CE 7F
58 F0 B7 CE 7F
58 F0 B7 CE 7F
58 F0 B7 CE 7F
58 F0 B7 CE 7F
58 F0 B7 CE 7F
58 F0 B7 CE 7F
58 F0 B7 CE 7F


In [51]:
poses = dict()
for size in [1, 2, 4, 8, 16, 32, 64]:
    print(f'size = {size}')
    poses[size] = dict()
    for pos in range(len(stacks[0]) - offset_7MB - 5*size):
        peaces = set()
        values = set()
        for stack in stacks:
            peaces.add(stack[pos: pos + 5*size])
            for i in range(5):
                values.add(stack[pos + i*size: pos + (i + 1)*size])
        if 2 <= len(peaces) <= len(stacks) and len(values) <= 4: # В этих переговорах было по 4 товара. Считаем вероятность полного совпадения двух переговоров исчезающе малой.
            poses[size][pos] = peaces
            print(pos)
            for peace in peaces:
                print('    ' + bytes_to_str(peace))

size = 1
size = 2
size = 4
size = 8
size = 16
size = 32
size = 64


In [44]:
'Видимо не на стеке'

'Видимо не на стеке'

In [50]:
keys = [
('0x7dcf1ea000', 4096, '/dev/ashmem/b390658b-3797-4724-bafe-266be4332e9d'),
('0x7e502e3000', 4096, '/dev/ashmem/b390658b-3797-4724-bafe-266be4332e9d'),
('0x7e5042c000', 4096, '/dev/ashmem/b390658b-3797-4724-bafe-266be4332e9d'),
('0x7e554d5000', 4096, '[vvar]')]
poses = dict()
for key in keys:
    poses[key] = dict()
    print(key)
    blocks = list()
    for dump in dumps:
        blocks.append(dump[key])
    for size in [1, 2, 4, 8, 16, 32, 64]:
        print(f'size = {size}')
        poses[key][size] = dict()
        for pos in range(len(blocks[0]) - 5*size):
            peaces = set()
            values = set()
            for block in blocks:
                peaces.add(block[pos: pos + 5*size])
                for i in range(5):
                    values.add(block[pos + i*size: pos + (i + 1)*size])
            if 2 <= len(peaces) <= len(blocks) and len(values) <= 4: # В этих переговорах было по 4 товара. Считаем вероятность полного совпадения двух переговоров исчезающе малой.
                poses[key][size][pos] = peaces
                print(pos)
                for peace in peaces:
                    print('    ' + bytes_to_str(peace))

('0x7dcf1ea000', 4096, '/dev/ashmem/b390658b-3797-4724-bafe-266be4332e9d')
size = 1
size = 2
size = 4
size = 8
size = 16
size = 32
size = 64
('0x7e502e3000', 4096, '/dev/ashmem/b390658b-3797-4724-bafe-266be4332e9d')
size = 1
size = 2
size = 4
size = 8
size = 16
size = 32
size = 64
('0x7e5042c000', 4096, '/dev/ashmem/b390658b-3797-4724-bafe-266be4332e9d')
size = 1
size = 2
size = 4
size = 8
size = 16
size = 32
size = 64
('0x7e554d5000', 4096, '[vvar]')
size = 1
size = 2
size = 4
size = 8
size = 16
size = 32
size = 64


In [ ]:
'''
Нужно проверить.
[anon:libc_malloc]
[32, 270532608]
'''